# This Project has the following sub-sections

### 1. Showing the map of Toronto

### 2. Fetching each neighborhoods of borough Scarborough and plotting them

### 3. Finding and sorting out 4 hotels in the borough 

### 4. Finding out the nearest cluster of - Coffee shop, fast food place, restraunt and a fancy dinner place- which              is least distance from hotel, and also, least distance from each other 

### 5. Analysing and selecting the best hotel in the borough

### 6. Finding out the user rating of every place there

### 7. Making a list of suggestions of the venues around the hotel with their respective distances

### 8. Bonus point: Recommending a couple of tour/travel spot nearby the hotel. 

# 1

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
import pandas as pd
d1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [11]:
d1.shape

(103, 3)

In [4]:
d1= d1[d1.Borough != 'Not assigned']
d1.shape

(103, 3)

In [5]:
d1= d1.replace(to_replace ="/", 
                 value =",", regex=True) 

In [9]:
d1.isnull().sum()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [10]:
d1["Neighborhood"].fillna(d1["Borough"], inplace = True) 

In [12]:
d1.sort_values('Postal code', ascending = True, inplace= True)

In [15]:
d1.reset_index(drop=True, inplace=True)

In [16]:
d1.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
dd= pd.read_csv('Geospatial_Coordinates.csv')

In [18]:
df= pd.concat([d1, dd], axis=1)

In [22]:
df.head(1)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353


In [21]:
df.drop(df.columns[[3]], axis= 1, inplace= True)

In [24]:
df.shape

(103, 5)

In [25]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# The map of Toronto

In [27]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
CLIENT_ID = 'FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK' 
CLIENT_SECRET = 'RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK
CLIENT_SECRET:RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE


In [31]:
data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
data.shape

(17, 5)

In [33]:
address = 'Scarborough, CA'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


# The map of Scarborough

In [34]:
map1 = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1)  
    
map1

In [55]:
dr= data

In [68]:
dr.drop(['Borough'], axis=1, inplace= True)

# The kist of all neighborhood in Scarborough

In [69]:
dr

,Neighborhood,Latitude,Longitude
0,"Malvern , Rouge",43.806686,-79.194353
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,Woburn,43.770992,-79.216917
4,Cedarbrae,43.773136,-79.239476
5,Scarborough Village,43.744734,-79.239476
6,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,"Birch Cliff , Cliffside West",43.692657,-79.264848


In [70]:
data.loc[0, 'Neighborhood']

'Malvern , Rouge'

In [71]:
neighborhood_latitude = data.loc[0, 'Latitude'] 
neighborhood_longitude = data.loc[0, 'Longitude'] 

neighborhood_name = data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern , Rouge are 43.806686299999996, -79.19435340000001.


In [72]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

In [73]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea2ec23da9e14001b643f10'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6718c2f964a5203f3a2be3',
       'name': "Harvey's",
       'location': {'address': '853 Milner Ave',
        'crossStreet': 'at Morningside Ave',
        'lat': 43.80002047583462,
        'lng': -79.19830662127322,
        'labeledLatLngs': [{'label':

In [74]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [75]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harvey's,Restaurant,43.800020,-79.198307
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
3,RBC Royal Bank,Bank,43.798782,-79.197090
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777


# Here we are showing a list of all venues in Malvern , Rouge within 1000m 

In [78]:
nearby_venues

,name,categories,lat,lng
0,Harvey's,Restaurant,43.800020,-79.198307
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
3,RBC Royal Bank,Bank,43.798782,-79.197090
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
5,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
6,Tim Hortons,Coffee Shop,43.802000,-79.198169
7,Lee Valley,Hobby Shop,43.803161,-79.199681
8,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
9,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296


# Monte Carlo Inn Proximity 

### Coffee Point

In [115]:
address = 'Monte Carlo Inn, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6197126 -79.57203464983789


In [119]:
search_query = 'Coffee'
radius = 1000
print(search_query + ' .... OK!')

Coffee .... OK!


In [120]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6197126,-79.57203464983789&v=20180605&query=Coffee&radius=1000&limit=100'

In [121]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea333c669babe001b6223bc'},
 'response': {'venues': [{'id': '524a147b11d2985983efdcb6',
    'name': 'Starbucks',
    'location': {'address': '1560 Dundas Street East',
     'crossStreet': 'Dixie Road',
     'lat': 43.612016479990174,
     'lng': -79.57858300126318,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.612016479990174,
       'lng': -79.57858300126318}],
     'distance': 1006,
     'postalCode': 'L4X 1L4',
     'cc': 'CA',
     'city': 'Mississauga',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1560 Dundas Street East (Dixie Road)',
      'Mississauga ON L4X 1L4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587754022',


In [122]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",False,524a147b11d2985983efdcb6,1560 Dundas Street East,CA,Mississauga,Canada,Dixie Road,1006,"[1560 Dundas Street East (Dixie Road), Mississauga ON L4X 1L4, Canada]","[{'label': 'display', 'lat': 43.612016479990174, 'lng': -79.57858300126318}]",43.612016,-79.578583,L4X 1L4,ON,Starbucks,v-1587754022


In [124]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Starbucks,Coffee Shop,1560 Dundas Street East,CA,Mississauga,Canada,Dixie Road,1006,"[1560 Dundas Street East (Dixie Road), Mississauga ON L4X 1L4, Canada]","[{'label': 'display', 'lat': 43.612016479990174, 'lng': -79.57858300126318}]",43.612016,-79.578583,L4X 1L4,ON,524a147b11d2985983efdcb6


In [125]:
dataframe_filtered.name

0    Starbucks
Name: name, dtype: object

In [126]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [127]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [128]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [129]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Brunch

In [137]:
search_query = 'Restaurant'
radius = 100
print(search_query + ' .... OK!')

Restaurant .... OK!


In [138]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6197126,-79.57203464983789&v=20180605&query=Restaurant&radius=100&limit=100'

In [139]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea338a21835dd05eb3f1a3c'},
 'response': {'venues': [{'id': '4bdf6678e75c0f470617cb03',
    'name': 'Silk Road Restaurant 丝绸之路',
    'location': {'address': '1852 Dundas St E',
     'crossStreet': 'at Wharton Way',
     'lat': 43.61927817099883,
     'lng': -79.57303110058828,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.61927817099883,
       'lng': -79.57303110058828}],
     'distance': 93,
     'postalCode': 'L4X 1L9',
     'cc': 'CA',
     'city': 'Mississauga',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1852 Dundas St E (at Wharton Way)',
      'Mississauga ON L4X 1L9',
      'Canada']},
    'categories': [{'id': '52af3b913cf9994f4e043c06',
      'name': 'Xinjiang Restaurant',
      'pluralName': 'Xinjiang Restaurants',
      'shortName': 'Xinjiang',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v

In [140]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52af3b913cf9994f4e043c06', 'name': 'Xinjiang Restaurant', 'pluralName': 'Xinjiang Restaurants', 'shortName': 'Xinjiang', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4bdf6678e75c0f470617cb03,1852 Dundas St E,CA,Mississauga,Canada,at Wharton Way,93,"[1852 Dundas St E (at Wharton Way), Mississauga ON L4X 1L9, Canada]","[{'label': 'display', 'lat': 43.61927817099883, 'lng': -79.57303110058828}]",43.619278,-79.573031,L4X 1L9,ON,Silk Road Restaurant 丝绸之路,v-1587755359


In [142]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Silk Road Restaurant 丝绸之路,Xinjiang Restaurant,1852 Dundas St E,CA,Mississauga,Canada,at Wharton Way,93,"[1852 Dundas St E (at Wharton Way), Mississauga ON L4X 1L9, Canada]","[{'label': 'display', 'lat': 43.61927817099883, 'lng': -79.57303110058828}]",43.619278,-79.573031,L4X 1L9,ON,4bdf6678e75c0f470617cb03


In [143]:
dataframe_filtered.name

0    Silk Road Restaurant 丝绸之路
Name: name, dtype: object

In [144]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [145]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [146]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [147]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# Bar and Dinner

In [160]:
search_query = 'Bar'
radius = 700
print(search_query + ' .... OK!')

Bar .... OK!


In [161]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6197126,-79.57203464983789&v=20180605&query=Bar&radius=700&limit=100'

In [162]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea33a40006dce001bccf442'},
 'response': {'venues': [{'id': '4ad4c05df964a5202cf620e3',
    'name': 'Second Floor Lounge Bar & Grill',
    'location': {'address': '1565 Dundas Street East',
     'lat': 43.61296706405266,
     'lng': -79.57691269940739,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.61296706405266,
       'lng': -79.57691269940739}],
     'distance': 847,
     'postalCode': 'L4X 1L5',
     'cc': 'CA',
     'city': 'Mississauga',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1565 Dundas Street East',
      'Mississauga ON L4X 1L5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d121941735',
      'name': 'Lounge',
      'pluralName': 'Lounges',
      'shortName': 'Lounge',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587755900',
    'hasPerk': False}]}}

In [163]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d121941735', 'name': 'Lounge', 'pluralName': 'Lounges', 'shortName': 'Lounge', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_', 'suffix': '.png'}, 'primary': True}]",False,4ad4c05df964a5202cf620e3,1565 Dundas Street East,CA,Mississauga,Canada,847,"[1565 Dundas Street East, Mississauga ON L4X 1L5, Canada]","[{'label': 'display', 'lat': 43.61296706405266, 'lng': -79.57691269940739}]",43.612967,-79.576913,L4X 1L5,ON,Second Floor Lounge Bar & Grill,v-1587755900


In [164]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Second Floor Lounge Bar & Grill,Lounge,1565 Dundas Street East,CA,Mississauga,Canada,847,"[1565 Dundas Street East, Mississauga ON L4X 1L5, Canada]","[{'label': 'display', 'lat': 43.61296706405266, 'lng': -79.57691269940739}]",43.612967,-79.576913,L4X 1L5,ON,4ad4c05df964a5202cf620e3


In [165]:
dataframe_filtered.name

0    Second Floor Lounge Bar & Grill
Name: name, dtype: object

In [166]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [167]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [168]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [169]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# Days Inn by Wyndham

### Coffee point

In [171]:
address = 'Days Inn, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6672363 -79.313387


In [187]:
search_query = 'Coffee'
radius = 150
print(search_query + ' .... OK!')

Coffee .... OK!


In [188]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6672363,-79.313387&v=20180605&query=Coffee&radius=150&limit=100'

In [189]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea33dff71c428001f7ba684'},
 'response': {'venues': [{'id': '4d87a6a440a7a35d205a4cbe',
    'name': 'Coffee Time',
    'location': {'address': '1614 queen street east',
     'crossStreet': 'Coxwell',
     'lat': 43.667814,
     'lng': -79.311798,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.667814,
       'lng': -79.311798}],
     'distance': 143,
     'postalCode': 'M4L 1S5',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1614 queen street east (Coxwell)',
      'Toronto ON M4L 1S5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587756815',
    'hasPerk': False}]}}

In [190]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}]",False,4d87a6a440a7a35d205a4cbe,1614 queen street east,CA,Toronto,Canada,Coxwell,143,"[1614 queen street east (Coxwell), Toronto ON M4L 1S5, Canada]","[{'label': 'display', 'lat': 43.667814, 'lng': -79.311798}]",43.667814,-79.311798,M4L 1S5,ON,Coffee Time,v-1587756815


In [191]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Coffee Time,Café,1614 queen street east,CA,Toronto,Canada,Coxwell,143,"[1614 queen street east (Coxwell), Toronto ON M4L 1S5, Canada]","[{'label': 'display', 'lat': 43.667814, 'lng': -79.311798}]",43.667814,-79.311798,M4L 1S5,ON,4d87a6a440a7a35d205a4cbe


In [192]:
dataframe_filtered.name

0    Coffee Time
Name: name, dtype: object

In [193]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [194]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [195]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [196]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Brunch point

In [203]:
search_query = 'Restaurant'
radius = 600
print(search_query + ' .... OK!')

Restaurant .... OK!


In [204]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6672363,-79.313387&v=20180605&query=Restaurant&radius=600&limit=100'

In [205]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea33fe40be7b4001b63d8d7'},
 'response': {'venues': [{'id': '4b58eeb6f964a520e37328e3',
    'name': 'Delina Restaurant',
    'location': {'address': '1891 Queen St E',
     'crossStreet': 'Woodbine',
     'lat': 43.6688671759284,
     'lng': -79.3054042721163,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.6688671759284,
       'lng': -79.3054042721163}],
     'distance': 667,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1891 Queen St E (Woodbine)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d115941735',
      'name': 'Middle Eastern Restaurant',
      'pluralName': 'Middle Eastern Restaurants',
      'shortName': 'Middle Eastern',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587757032',
    'hasPerk': False}]}}

In [206]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d115941735', 'name': 'Middle Eastern Restaurant', 'pluralName': 'Middle Eastern Restaurants', 'shortName': 'Middle Eastern', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",False,4b58eeb6f964a520e37328e3,1891 Queen St E,CA,Toronto,Canada,Woodbine,667,"[1891 Queen St E (Woodbine), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6688671759284, 'lng': -79.3054042721163}]",43.668867,-79.305404,ON,Delina Restaurant,v-1587757032


In [207]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Delina Restaurant,Middle Eastern Restaurant,1891 Queen St E,CA,Toronto,Canada,Woodbine,667,"[1891 Queen St E (Woodbine), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6688671759284, 'lng': -79.3054042721163}]",43.668867,-79.305404,ON,4b58eeb6f964a520e37328e3


In [208]:
dataframe_filtered.name

0    Delina Restaurant
Name: name, dtype: object

In [209]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [210]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [211]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [212]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Bar and Dinner

In [235]:
search_query = 'Bar'
radius = 220
print(search_query + ' .... OK!')

Bar .... OK!


In [236]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.6672363,-79.313387&v=20180605&query=Bar&radius=220&limit=100'

In [237]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea3414b98205d74787490a2'},
 'response': {'venues': [{'id': '4c3a24823849c9286477c2b1',
    'name': 'The Emerald Bar & Grill',
    'location': {'address': '1628 Queen St. East',
     'lat': 43.666548,
     'lng': -79.31596,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.666548,
       'lng': -79.31596}],
     'distance': 220,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1628 Queen St. East', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'shortName': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587757654',
    'hasPerk': False}]}}

In [238]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d116941735', 'name': 'Bar', 'pluralName': 'Bars', 'shortName': 'Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}, 'primary': True}]",False,4c3a24823849c9286477c2b1,1628 Queen St. East,CA,Toronto,Canada,220,"[1628 Queen St. East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.666548, 'lng': -79.31596}]",43.666548,-79.31596,ON,The Emerald Bar & Grill,v-1587757654


In [239]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,The Emerald Bar & Grill,Bar,1628 Queen St. East,CA,Toronto,Canada,220,"[1628 Queen St. East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.666548, 'lng': -79.31596}]",43.666548,-79.31596,ON,4c3a24823849c9286477c2b1


In [240]:
dataframe_filtered.name

0    The Emerald Bar & Grill
Name: name, dtype: object

In [241]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [242]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [243]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [244]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# Delta Hotels by Marriott

### Coffee point

In [246]:
address = 'Delta hotels, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.643082 -79.38387181137503


In [256]:
search_query = 'Coffee'
radius = 150
print(search_query + ' .... OK!')

Coffee .... OK!


In [257]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.643082,-79.38387181137503&v=20180605&query=Coffee&radius=150&limit=100'

In [258]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea3419edd0f85001bea0e7e'},
 'response': {'venues': [{'id': '557f05ca498ec78ac7b29315',
    'name': "Balzac's Coffee",
    'location': {'address': '7 Station St',
     'crossStreet': 'at SkyWalk',
     'lat': 43.644372584148364,
     'lng': -79.38306470027423,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.644372584148364,
       'lng': -79.38306470027423}],
     'distance': 157,
     'postalCode': 'M5J 1C3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['7 Station St (at SkyWalk)',
      'Toronto ON M5J 1C3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587757925',
    'hasPerk': False}]}}

In [259]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",False,557f05ca498ec78ac7b29315,7 Station St,CA,Toronto,Canada,at SkyWalk,157,"[7 Station St (at SkyWalk), Toronto ON M5J 1C3, Canada]","[{'label': 'display', 'lat': 43.644372584148364, 'lng': -79.38306470027423}]",43.644373,-79.383065,M5J 1C3,ON,Balzac's Coffee,v-1587757925


In [260]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Balzac's Coffee,Coffee Shop,7 Station St,CA,Toronto,Canada,at SkyWalk,157,"[7 Station St (at SkyWalk), Toronto ON M5J 1C3, Canada]","[{'label': 'display', 'lat': 43.644372584148364, 'lng': -79.38306470027423}]",43.644373,-79.383065,M5J 1C3,ON,557f05ca498ec78ac7b29315


In [261]:
dataframe_filtered.name

0    Balzac's Coffee
Name: name, dtype: object

In [262]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [263]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [264]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [265]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Brunch Point

In [275]:
search_query = 'Restaurant'
radius = 100
print(search_query + ' .... OK!')

Restaurant .... OK!


In [276]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.643082,-79.38387181137503&v=20180605&query=Restaurant&radius=100&limit=100'

In [277]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea34343a2e538001b6c9ec3'},
 'response': {'venues': [{'id': '4c0163083f0a9c741d4fa0ec',
    'name': 'Subway',
    'location': {'address': '127 Bremner Boulevard',
     'crossStreet': 'at Lower Simcoe St',
     'lat': 43.642485063503074,
     'lng': -79.38317879081399,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.642485063503074,
       'lng': -79.38317879081399}],
     'distance': 86,
     'postalCode': 'M5J 3A8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['127 Bremner Boulevard (at Lower Simcoe St)',
      'Toronto ON M5J 3A8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c5941735',
      'name': 'Sandwich Place',
      'pluralName': 'Sandwich Places',
      'shortName': 'Sandwiches',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587758110',
  

In [278]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'Sandwich Place', 'pluralName': 'Sandwich Places', 'shortName': 'Sandwiches', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_', 'suffix': '.png'}, 'primary': True}]",False,4c0163083f0a9c741d4fa0ec,127 Bremner Boulevard,CA,Toronto,Canada,at Lower Simcoe St,86,"[127 Bremner Boulevard (at Lower Simcoe St), Toronto ON M5J 3A8, Canada]","[{'label': 'display', 'lat': 43.642485063503074, 'lng': -79.38317879081399}]",43.642485,-79.383179,M5J 3A8,ON,Subway,v-1587758110


In [279]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Subway,Sandwich Place,127 Bremner Boulevard,CA,Toronto,Canada,at Lower Simcoe St,86,"[127 Bremner Boulevard (at Lower Simcoe St), Toronto ON M5J 3A8, Canada]","[{'label': 'display', 'lat': 43.642485063503074, 'lng': -79.38317879081399}]",43.642485,-79.383179,M5J 3A8,ON,4c0163083f0a9c741d4fa0ec


In [280]:
dataframe_filtered.name

0    Subway
Name: name, dtype: object

In [281]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [282]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [283]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [284]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Bar and Dineer Point

In [291]:
search_query = 'bar'
radius = 50
print(search_query + ' .... OK!')

bar .... OK!


In [292]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.643082,-79.38387181137503&v=20180605&query=bar&radius=50&limit=100'

In [293]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea34554b57e88001bacdfca'},
 'response': {'venues': [{'id': '545bef71498eddc2599b086a',
    'name': 'SOCO Kitchen+Bar',
    'location': {'address': '75 Lower Simcoe Street',
     'crossStreet': 'Bremner',
     'lat': 43.64303873929366,
     'lng': -79.38380850724118,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64303873929366,
       'lng': -79.38380850724118}],
     'distance': 7,
     'postalCode': 'M5J 3A6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['75 Lower Simcoe Street (Bremner)',
      'Toronto ON M5J 3A6',
      'Canada']},
    'categories': [{'id': '52e81612bcbc57f1066b79f1',
      'name': 'Bistro',
      'pluralName': 'Bistros',
      'shortName': 'Bistro',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587758341',
    'hasPerk': False}]}}

In [294]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'Bistro', 'pluralName': 'Bistros', 'shortName': 'Bistro', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",False,545bef71498eddc2599b086a,75 Lower Simcoe Street,CA,Toronto,Canada,Bremner,7,"[75 Lower Simcoe Street (Bremner), Toronto ON M5J 3A6, Canada]","[{'label': 'display', 'lat': 43.64303873929366, 'lng': -79.38380850724118}]",43.643039,-79.383809,M5J 3A6,ON,SOCO Kitchen+Bar,v-1587758341


In [295]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,SOCO Kitchen+Bar,Bistro,75 Lower Simcoe Street,CA,Toronto,Canada,Bremner,7,"[75 Lower Simcoe Street (Bremner), Toronto ON M5J 3A6, Canada]","[{'label': 'display', 'lat': 43.64303873929366, 'lng': -79.38380850724118}]",43.643039,-79.383809,M5J 3A6,ON,545bef71498eddc2599b086a


In [296]:
dataframe_filtered.name

0    SOCO Kitchen+Bar
Name: name, dtype: object

In [297]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [298]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [299]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [300]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# knights Inn 

### Coffee shop

In [301]:
address = 'Knights Inn, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7778225 -79.17281405263158


In [302]:
search_query = 'Coffee'
radius = 1000
print(search_query + ' .... OK!')

Coffee .... OK!


In [303]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.7778225,-79.17281405263158&v=20180605&query=Coffee&radius=1000&limit=100'

In [304]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea34841b4b684001b125470'},
 'response': {'venues': [{'id': '5206752511d2e01992c02a8b',
    'name': 'Coffee Time',
    'location': {'lat': 43.783636436107955,
     'lng': -79.17030424453813,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.783636436107955,
       'lng': -79.17030424453813}],
     'distance': 677,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d123951735',
      'name': 'Smoke Shop',
      'pluralName': 'Smoke Shops',
      'shortName': 'Smoke Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/tobacco_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587759410',
    'hasPerk': False}]}}

In [305]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.cc,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,name,referralId
0,"[{'id': '4bf58dd8d48988d123951735', 'name': 'Smoke Shop', 'pluralName': 'Smoke Shops', 'shortName': 'Smoke Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/tobacco_', 'suffix': '.png'}, 'primary': True}]",False,5206752511d2e01992c02a8b,CA,Canada,677,[Canada],"[{'label': 'display', 'lat': 43.783636436107955, 'lng': -79.17030424453813}]",43.783636,-79.170304,Coffee Time,v-1587759410


In [306]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,cc,country,distance,formattedAddress,labeledLatLngs,lat,lng,id
0,Coffee Time,Smoke Shop,CA,Canada,677,[Canada],"[{'label': 'display', 'lat': 43.783636436107955, 'lng': -79.17030424453813}]",43.783636,-79.170304,5206752511d2e01992c02a8b


In [307]:
dataframe_filtered.name

0    Coffee Time
Name: name, dtype: object

In [308]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [309]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [310]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [311]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Brunch Point

In [312]:
search_query = 'Restaurant'
radius = 1000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [313]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.7778225,-79.17281405263158&v=20180605&query=Restaurant&radius=1000&limit=100'

In [314]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea348ff95feaf001bac2b5c'},
 'response': {'venues': [{'id': '4be6c179d4f7c9b665042720',
    'name': "Ted's Restaurant",
    'location': {'address': '404 Old Kingston Rd.',
     'lat': 43.78446796744621,
     'lng': -79.1692004571639,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.78446796744621,
       'lng': -79.1692004571639}],
     'distance': 794,
     'cc': 'CA',
     'city': 'Scarborough',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['404 Old Kingston Rd.', 'Scarborough ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d143941735',
      'name': 'Breakfast Spot',
      'pluralName': 'Breakfast Spots',
      'shortName': 'Breakfast',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587759654',
    'hasPerk': False}]}}

In [315]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d143941735', 'name': 'Breakfast Spot', 'pluralName': 'Breakfast Spots', 'shortName': 'Breakfast', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_', 'suffix': '.png'}, 'primary': True}]",False,4be6c179d4f7c9b665042720,404 Old Kingston Rd.,CA,Scarborough,Canada,794,"[404 Old Kingston Rd., Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.78446796744621, 'lng': -79.1692004571639}]",43.784468,-79.1692,ON,Ted's Restaurant,v-1587759654


In [316]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Ted's Restaurant,Breakfast Spot,404 Old Kingston Rd.,CA,Scarborough,Canada,794,"[404 Old Kingston Rd., Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.78446796744621, 'lng': -79.1692004571639}]",43.784468,-79.1692,ON,4be6c179d4f7c9b665042720


In [317]:
dataframe_filtered.name

0    Ted's Restaurant
Name: name, dtype: object

In [318]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [319]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [320]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [321]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Bar and Dinner 

In [325]:
search_query = 'Bar'
radius = 500
print(search_query + ' .... OK!')

Bar .... OK!


In [326]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.7778225,-79.17281405263158&v=20180605&query=Bar&radius=500&limit=100'

In [327]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea349ba95feaf001bae0462'},
 'response': {'venues': [{'id': '5720ef69498ee3255f58cf41',
    'name': 'The Lab Barbell & Social Club',
    'location': {'address': '4630 Kingston Rd',
     'lat': 43.774775,
     'lng': -79.179131,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.774775,
       'lng': -79.179131}],
     'distance': 610,
     'postalCode': 'M1E 4Z4',
     'cc': 'CA',
     'city': 'Scarborough',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4630 Kingston Rd',
      'Scarborough ON M1E 4Z4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587759841',
    'hasPerk': False}]}}

In [328]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d175941735', 'name': 'Gym / Fitness Center', 'pluralName': 'Gyms or Fitness Centers', 'shortName': 'Gym / Fitness', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",False,5720ef69498ee3255f58cf41,4630 Kingston Rd,CA,Scarborough,Canada,610,"[4630 Kingston Rd, Scarborough ON M1E 4Z4, Canada]","[{'label': 'display', 'lat': 43.774775, 'lng': -79.179131}]",43.774775,-79.179131,M1E 4Z4,ON,The Lab Barbell & Social Club,v-1587759841


In [329]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Lab Barbell & Social Club,Gym / Fitness Center,4630 Kingston Rd,CA,Scarborough,Canada,610,"[4630 Kingston Rd, Scarborough ON M1E 4Z4, Canada]","[{'label': 'display', 'lat': 43.774775, 'lng': -79.179131}]",43.774775,-79.179131,M1E 4Z4,ON,5720ef69498ee3255f58cf41


In [330]:
dataframe_filtered.name

0    The Lab Barbell & Social Club
Name: name, dtype: object

In [331]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [332]:
venue_id = '4dad8ab9043718a63e33e294' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dad8ab9043718a63e33e294?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&v=20180605'

In [333]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4dad8ab9043718a63e33e294',
 'name': 'Coffee Time',
 'contact': {},
 'location': {'address': '2195 Midland Ave',
  'crossStreet': 'at Progress Ave',
  'lat': 43.773307723529335,
  'lng': -79.27289954044328,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.773307723529335,
    'lng': -79.27289954044328}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['2195 Midland Ave (at Progress Ave)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-time/4dad8ab9043718a63e33e294',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
 

In [334]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Now we can create a matrix data frame to show which hotel has the least distances from a coffee shop, a brunch point and a dinner place. 

In [335]:
data = [['Monte Carlo Inn', 'Warden Ave', 1006, 93, 847], ['Days inn', 'Kingston Rd', 143, 667, 220], ['Delta Hotels', 'Kennedy Rd', 157, 86, 7], ['Knights Inn', 'Kingston Rd', 677, 794, 610]]

In [336]:
hotel = pd.DataFrame(data, columns = ['Hotel_name', 'Address', 'Coffee shop distance', 'Brunch distance', 'Bar Dinner']) 

In [337]:
hotel

,Hotel_name,Address,Coffee shop distance,Brunch distance,Bar Dinner
0,Monte Carlo Inn,Warden Ave,1006,93,847
1,Days inn,Kingston Rd,143,667,220
2,Delta Hotels,Kennedy Rd,157,86,7
3,Knights Inn,Kingston Rd,677,794,610


## Calling a function to sum up all the distances from the hotel 

In [338]:
def cal(x,y,z):
    total= x+y+z
    return total
    

In [348]:
dg= cal(hotel['Coffee shop distance'], hotel['Brunch distance'], hotel['Bar Dinner'])

In [349]:
dg

0    1946
1    1030
2    250 
3    2081
dtype: int64

In [357]:
print('The least distances for the coffee shop, brunch and dinner place is {} and the hotel is Delta Hotels'.format(min(dg)))

The least distances for the coffee shop, brunch and dinner place is 250 and the hotel is Delta Hotels


## As we can see from the function, the best option here is Delta house by Marriott which holds the best proximity from each of our interest- Coffee shop, brunch point and a Bar and dinner place

.

# BONUS POINT

## Now we are going to suggest 2 tourist places of interest near the Delta house hotel 

.

In [3]:
address = 'Delta house, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.643082 -79.38387181137503


In [5]:
search_query = 'Tourist'
radius = 1000
print(search_query + ' .... OK!')

Tourist .... OK!


In [9]:
LIMIT= 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FXWG412PGZMV0OTDDNYEC2XP2TGWP1ZTH5JD1RCML2OL5CLK&client_secret=RCZUNJDEY0MD0TH0KA5E3EDRLAF1T35JO5GSZNXQKZ0JX4PE&ll=43.643082,-79.38387181137503&v=20180605&query=Tourist&radius=1000&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea42ff283525f001b281296'},
 'response': {'venues': [{'id': '4c29077f3492a593d04cb828',
    'name': 'Local Resident and Tourist Kettle',
    'location': {'address': 'Queen St West',
     'crossStreet': 'Spadina',
     'lat': 43.64896344297954,
     'lng': -79.39682006835938,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64896344297954,
       'lng': -79.39682006835938}],
     'distance': 1231,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Queen St West (Spadina)', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d126941735',
      'name': 'Government Building',
      'pluralName': 'Government Buildings',
      'shortName': 'Government',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587818546',
    'hasPerk': False},
   {'id': '5b119a225

In [11]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,4c29077f3492a593d04cb828,Queen St West,CA,Toronto,Canada,Spadina,1231,"[Queen St West (Spadina), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64896344297954...",43.648963,-79.396820,NaN,ON,Local Resident and Tourist Kettle,v-1587818546
1,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",False,5b119a2251950e002c1b7c4b,481 University Ave.,CA,Toronto,Canada,NaN,896,"[481 University Ave., Toronto ON M5G 2E9, Canada]","[{'label': 'display', 'lat': 43.650894, 'lng':...",43.650894,-79.386563,M5G 2E9,ON,Japan National Tourist Organization - JNTO,v-1587818546


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Local Resident and Tourist Kettle,Government Building,Queen St West,CA,Toronto,Canada,Spadina,1231,"[Queen St West (Spadina), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64896344297954...",43.648963,-79.396820,NaN,ON,4c29077f3492a593d04cb828
1,Japan National Tourist Organization - JNTO,Tourist Information Center,481 University Ave.,CA,Toronto,Canada,NaN,896,"[481 University Ave., Toronto ON M5G 2E9, Canada]","[{'label': 'display', 'lat': 43.650894, 'lng':...",43.650894,-79.386563,M5G 2E9,ON,5b119a2251950e002c1b7c4b


In [13]:
dataframe_filtered.name

0             Local Resident and Tourist Kettle
1    Japan National Tourist Organization - JNTO
Name: name, dtype: object

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

## As we can see the places are- 

### Local Resident and Tourist Kettle
### Tourist Information Center